##### Support Request Generator

Generate generic support requests from delivered orders and assign repeat user IDs.

In [ ]:
CATALOG = dbutils.widgets.get("CATALOG")
SUPPORT_RATE = float(dbutils.widgets.get("SUPPORT_RATE"))
LLM_MODEL = dbutils.widgets.get("LLM_MODEL")

In [ ]:
from pyspark.sql import functions as F

spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.support")
spark.sql(f"CREATE VOLUME IF NOT EXISTS {CATALOG}.support.checkpoints")

# Deterministic repeatable user IDs from order_id hash to ensure repeated users.
base_requests = (
    spark.readStream
    .table(f"{CATALOG}.lakeflow.all_events")
    .filter(F.col("event_type") == "delivered")
    .filter(F.rand() < SUPPORT_RATE)
    .withColumn("support_request_id", F.expr("uuid()"))
    .withColumn("hash_bucket", F.abs(F.hash(F.col("order_id"))) % F.lit(25))
    .withColumn("user_id", F.concat(F.lit("user-"), F.format_string("%03d", F.col("hash_bucket"))))
    .withColumn(
        "request_type",
        F.expr(
            """
            CASE
              WHEN hash_bucket % 5 = 0 THEN 'delivery_delay'
              WHEN hash_bucket % 5 = 1 THEN 'missing_items'
              WHEN hash_bucket % 5 = 2 THEN 'food_quality'
              WHEN hash_bucket % 5 = 3 THEN 'billing'
              ELSE 'general_support'
            END
            """
        ),
    )
    .withColumn(
        "request_text",
        F.expr(
            """
            CASE
              WHEN request_type = 'delivery_delay' THEN 'My delivery arrived much later than expected. Can you help with a credit or refund?'
              WHEN request_type = 'missing_items' THEN 'My order was missing one or more items. Please review and advise next steps.'
              WHEN request_type = 'food_quality' THEN 'I had a quality issue with my food. I would like support and possible compensation.'
              WHEN request_type = 'billing' THEN 'I think there is a billing issue on my recent order. Can you verify the charge?'
              ELSE 'I need help with a recent order. Please review my request and let me know what you can do.'
            END
            """
        ),
    )
    .select(
        "support_request_id",
        "user_id",
        "order_id",
        F.current_timestamp().alias("ts"),
        "request_type",
        "request_text",
        F.lit("generator-v1").alias("generated_by"),
    )
)

spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {CATALOG}.support.raw_support_requests (
      support_request_id STRING,
      user_id STRING,
      order_id STRING,
      ts TIMESTAMP,
      request_type STRING,
      request_text STRING,
      generated_by STRING
    )
    """
)

base_requests.writeStream \
    .option("checkpointLocation", f"/Volumes/{CATALOG}/support/checkpoints/support_request_generator") \
    .trigger(availableNow=True) \
    .table(f"{CATALOG}.support.raw_support_requests")